In [ ]:
from astroquery.vizier import Vizier
vizier = Vizier()

from astropy.coordinates import SkyCoord
import astropy.units as u

import matplotlib.pyplot as plt
import numpy as np
import numpy.ma as ma
import pandas as pd

## Query the LMC variable dataset
Directly get accesss from Vizier with the catalogue ID: `J/A+A/430/603/variables`

In [ ]:
vizier = vizier(columns=["RAJ2000", "DEJ2000", "VType", "Per","<Bmag>", "<Vmag>", "<Imag>","Notes"])
vizier.ROW_LIMIT = 9999

In [ ]:
LMC_rr = vizier.get_catalogs("J/A+A/430/603/variables")

### Select the RR Lyrae stars in type **ab**

In [ ]:
LMC_rr0 = []

for i in range(len(LMC_rr[0])):
    if LMC_rr[0][i]["VType"] == "ab" and LMC_rr[0][i]["Notes"] != "Incomplete" and LMC_rr[0][i]["Notes"] != "Blend":
        
        # Convert the coordinate string to astropy Skycoord
        obs_coord = SkyCoord(
                             ra=LMC_rr[0][i]["RAJ2000"], dec=LMC_rr[0][i]["DEJ2000"], 
                             unit=(u.hourangle, u.deg), 
                             frame='icrs'
                            )
        
        LMC_rr0.append({
            'Coordinate' : obs_coord,
                     'P' : LMC_rr[0][i]["Per"],
                 'obs_B' : LMC_rr[0][i]["__Bmag_"],
                 'obs_V' : LMC_rr[0][i]["__Vmag_"],
                 'obs_I' : LMC_rr[0][i]["__Imag_"],
        })

RR0 samples: 94

### Select the RR Lyrae stars in type c

In [ ]:
LMC_rr1 = []

for i in range(len(LMC_rr[0])):
    if LMC_rr[0][i]["VType"] == "c" and LMC_rr[0][i]["Notes"] != "Incomplete" and LMC_rr[0][i]["Notes"] != "Blend":
        
        # Convert the coordinate string to astropy Skycoord
        obs_coord = SkyCoord(
                             ra=LMC_rr[0][i]["RAJ2000"], dec=LMC_rr[0][i]["DEJ2000"], 
                             unit=(u.hourangle, u.deg), 
                             frame='icrs'
                            )
        
        LMC_rr1.append({
            'Coordinate' : obs_coord,
                     'P' : LMC_rr[0][i]["Per"],
                 'obs_B' : LMC_rr[0][i]["__Bmag_"],
                 'obs_V' : LMC_rr[0][i]["__Vmag_"],
                 'obs_I' : LMC_rr[0][i]["__Imag_"],
        })

RR1 samples: 54

## Extinction
From [Gorski et al. 2020](https://ui.adsabs.harvard.edu/abs/2020ApJ...889..179G/abstract) derived the Magellanic clouds [extinction maps](https://araucaria.camk.edu.pl/index.php/magellanic-clouds-extinction-maps/) with different spatial resolution of map (using 3 arcmin)
- V band:  
    Utilize the relationship $ A_V = R_V \times E(B-V) $. The authors use $ R_V = 3.1 $ ([Cardelli et al. 1989](https://ui.adsabs.harvard.edu/abs/1989ApJ...345..245C/abstract) extinction law)
- B band:  
    Use the definition of $E(B-V)$, which is $E(B-V) = A_B - A_V$
    
- I band:  
    From the extinction law in Cardelli et al., the ratio $A_I/A_V\approx 0.479$

To sum up, the extinction law for each band is:

$$\begin{aligned}
    A_B &= E(B-V) + A_V \\
    A_V &= 3.1 \cdot E(B-V) \\
    A_I &= 0.479 \cdot A_V
\end{aligned}$$

1. Read the reddening map
2. Match observational targets to the map:  
    - Use the angular distance formula to find the closest grid point in the reddening map:  
    $$
        d = \sqrt{(\text{RA}_\text{target} - \text{RA}_\text{grid})^2 + (\text{Dec}_\text{target} - \text{Dec}_\text{grid})^2}
    $$
    Select the grid point with smallest distance $d$
     - Using the [`Skycoord.match_to_catalog_sky`](https://docs.astropy.org/en/stable/api/astropy.coordinates.SkyCoord.html)
     
### Error propagation
#### Extinction $A_\lambda$
$$\begin{aligned}
    \sigma_{A_B} &= \sqrt{\sigma_{E(B-V)}^2 + (3.1\sigma_{E(B-V)})^2} \\
    \sigma_{A_V} &= 3.1 \cdot \sigma_{E(B-V)} \\
    \sigma_{A_I} &= 0.479 \cdot \sigma_{E(B-V)}
\end{aligned}$$
#### Corrected magnitude
$$\begin{aligned}
    \sigma_{B_\text{corr}} &= \sigma_{A_B} \\
    \sigma_{V_\text{corr}} &= \sigma_{A_V} \\
    \sigma_{I_\text{corr}} &= \sigma_{A_I}
\end{aligned}$$

In [ ]:
# read the downloaded reddening E(B-V) map
reddening_map = pd.read_csv(
                            "EBV_res3_lmc.txt", 
                            delim_whitespace=True, 
                            comment='#',  # Ignore comment lines
                            names=['RA', 'Dec', 'E_BV', 'Error', 'Sigma_RC', 'N_RC']
                           )

map_coords = SkyCoord(ra=reddening_map["RA"] * u.deg, dec=reddening_map['Dec'] * u.deg)

In [ ]:
R_V = 3.1

AI_AV_ratio = 0.479

#### RR0

In [ ]:
rr0_corrected = []

for data in LMC_rr0:
    
    # use SkyCoord.match_to_catalog_sky 
    idx, d2d, d3d = data['Coordinate'].match_to_catalog_sky(map_coords)
    
    # idx is the index to the catalog (reddening map)
    E_BV = reddening_map['E_BV'][idx]
    
    A_V = R_V * E_BV
    A_B = A_V + E_BV
    A_I = AI_AV_ratio * A_V
    
    error_E_BV = reddening_map['Error'][idx]
    
    B_error = np.sqrt( error_E_BV**2 + (3.1 * error_E_BV)**2 )
    V_error = 3.1 * error_E_BV
    I_error = 0.479 * error_E_BV
    
    B_corrected = data['obs_B'] - A_B
    V_corrected = data['obs_V'] - A_V
    I_corrected = data['obs_I'] - A_I
    
    rr0_corrected.append({
               'E_BV' : E_BV,
                'A_B' : A_B,
                'A_V' : A_V,
                'A_I' : A_I,
                  'P' : data['P'],
        'B_corrected' : B_corrected,
        'V_corrected' : V_corrected,
        'I_corrected' : I_corrected, 
            'B_error' : B_error,
            'V_error' : V_error,
            'I_error' : I_error,
    })

#### RR1

In [ ]:
rr1_corrected = []

for data in LMC_rr1:
    
    # use SkyCoord.match_to_catalog_sky 
    idx, d2d, d3d = data['Coordinate'].match_to_catalog_sky(map_coords)
    
    # idx is the index to the catalog (reddening map)
    E_BV = reddening_map['E_BV'][idx]
    
    A_V = R_V * E_BV
    A_B = A_V + E_BV
    A_I = AI_AV_ratio * A_V
    
    error_E_BV = reddening_map['Error'][idx]
    
    B_error = np.sqrt( error_E_BV**2 + (3.1 * error_E_BV)**2 )
    V_error = 3.1 * error_E_BV
    I_error = 0.479 * error_E_BV
    
    B_corrected = data['obs_B'] - A_B
    V_corrected = data['obs_V'] - A_V
    I_corrected = data['obs_I'] - A_I
    
    rr1_corrected.append({
               'E_BV' : E_BV,
                'A_B' : A_B,
                'A_V' : A_V,
                'A_I' : A_I,
                  'P' : data['P'],
        'B_corrected' : B_corrected,
        'V_corrected' : V_corrected,
        'I_corrected' : I_corrected, 
            'B_error' : B_error,
            'V_error' : V_error,
            'I_error' : I_error,
    })

### Pan-STARRS1 and Johnson conversion
See [Tory et al.](https://ui.adsabs.harvard.edu/abs/2012ApJ...750...99T/abstract) equation (6) and table (6)  
The quadratic transformation gives more accurate result (lower uncertainty):
$$
    y = A_0 + A_1 x + A_2 x^2
$$

|$y$|$x$|$A_0$|$A_1$|$A_2$|$\pm$|
|:-:|:-:|:-:|:-:|:-:|:-:|
|$(g_{P1}-B)$|$(B-V)$|$-0.108$|$-0.845$|$-0.032$|$0.011$|
|$(r_{P1}-V)$|$(B-V)$|$0.082$|$-0.462$|$0.041$|$0.025$|
|$(i_{P1}-I)$|$(B-V)$|$0.341$|$0.154$|$-0.025$|$0.012$|

$$\begin{aligned}
    g_{P1} - B &= -0.108 - 0.845(B-V) - 0.032(B-V)^2 \\
    r_{P1} - V &= 0.082 + 0.462(B-V) + 0.041(B-V)^2 \\
    i_{P1} - I &= 0.341 - 0.154(B-V) - 0.025(B-V)^2 \\
\end{aligned}$$

#### Error propagation

In [ ]:
uA_g = 0.011
uA_r = 0.025
uA_i = 0.012

#### RR0 conversion

In [ ]:
rr0_gri = {'gmag' : [], 'err_g' : [],
           'rmag' : [], 'err_r' : [],
           'imag' : [], 'err_i' : []}

rr0_P   = {'P_g' : [],
           'P_r' : [],
           'P_i' : []}

for data in rr0_corrected:
    B  = data['B_corrected']
    uB = data['B_error']
        
    V  = data['V_corrected']
    uV = data['V_error']
    
    I  = data['I_corrected']
    uI = data['I_error']
    
    x  = data['B_corrected'] - data['V_corrected']
    ux = np.sqrt( uB**2 + uV**2 )
    
    # convert to g, r band together
    if ma.getmask(B) == False and ma.getmask(V) == False:
        
        gP1 = -0.108 - 0.845 * x - 0.032 * x**2 + B
        rP1 =  0.082 - 0.462 * x + 0.041 * x**2 + V
        
        err_g = np.sqrt( uB**2 + uA_g**2 + (x * uA_g)**2 + (-0.845*ux)**2 + (x**2 * uA_g)**2 + (2*-0.032*x*ux)**2 )
        err_r = np.sqrt( uV**2 + uA_r**2 + (x * uA_r)**2 + (-0.462*ux)**2 + (x**2 * uA_r)**2 + (2* 0.041*x*ux)**2 )
        
        rr0_gri['gmag'].append(gP1)
        rr0_gri['rmag'].append(rP1)
        
        rr0_gri['err_g'].append(err_g)
        rr0_gri['err_r'].append(err_r)
        
        rr0_P['P_g'].append(data['P'])
        rr0_P['P_r'].append(data['P'])
        
    # convert to i band
    if ma.getmask(B) == False and ma.getmask(V) == False and ma.getmask(I) == False:
        
        iP1 = 0.341 - 0.154 * x - 0.025 * x**2 + I
        
        err_i = np.sqrt( uI**2 + uA_i**2 + (x * uA_i)**2 + (-0.154*ux)**2 + (x**2 * uA_i)**2 + (2*-0.025*x*ux)**2 )
        
        rr0_gri['imag'].append(iP1)
        
        rr0_gri['err_i'].append(err_i)
        
        rr0_P['P_i'].append(data['P'])

RR0 sample numbers:
 - g-band: 74
 - r-band: 74
 - i-band: 58

#### RR1 conversion

In [ ]:
rr1_gri = {'gmag' : [], 'err_g' : [],
           'rmag' : [], 'err_r' : [],
           'imag' : [], 'err_i' : []}

rr1_P   = {'P_g' : [],
           'P_r' : [],
           'P_i' : []}

for data in rr1_corrected:
    B  = data['B_corrected']
    uB = data['B_error']
        
    V  = data['V_corrected']
    uV = data['V_error']
    
    I  = data['I_corrected']
    uI = data['I_error']
    
    x  = data['B_corrected'] - data['V_corrected']
    ux = np.sqrt( uB**2 + uV**2 )
    
    # convert to g, r band together
    if ma.getmask(B) == False and ma.getmask(V) == False:
        
        gP1 = -0.108 - 0.845 * x - 0.032 * x**2 + B
        rP1 =  0.082 - 0.462 * x + 0.041 * x**2 + V
        
        err_g = np.sqrt( uB**2 + uA_g**2 + (x * uA_g)**2 + (-0.845*ux)**2 + (x**2 * uA_g)**2 + (2*-0.032*x*ux)**2 )
        err_r = np.sqrt( uV**2 + uA_r**2 + (x * uA_r)**2 + (-0.462*ux)**2 + (x**2 * uA_r)**2 + (2* 0.041*x*ux)**2 )
        
        rr1_gri['gmag'].append(gP1)
        rr1_gri['rmag'].append(rP1)
        
        rr1_gri['err_g'].append(err_g)
        rr1_gri['err_r'].append(err_r)
        
        rr1_P['P_g'].append(data['P'])
        rr1_P['P_r'].append(data['P'])
        
    # convert to i band
    if ma.getmask(B) == False and ma.getmask(V) == False and ma.getmask(I) == False:
        
        iP1 = 0.341 - 0.154 * x - 0.025 * x**2 + I
        
        err_i = np.sqrt( uI**2 + uA_i**2 + (x * uA_i)**2 + (-0.154*ux)**2 + (x**2 * uA_i)**2 + (2*-0.025*x*ux)**2 )
        
        rr1_gri['imag'].append(iP1)
        
        rr1_gri['err_i'].append(err_i)
        
        rr1_P['P_i'].append(data['P'])

RR0 sample numbers:
 - g-band: 32
 - r-band: 32
 - i-band: 31

## Distance modulus $\mu = m - M$

### The distance modulus from old work
From [Clementini et al. 2003](https://ui.adsabs.harvard.edu/abs/2003AJ....125.1309C/abstract), the author used 3 method to estimated the absolute magnitude:  

|Method|$M_V$|$\mu$|
|:-|:-:|:-:|
|Baade-Wesselink method|$0.68 \pm 0.15$|$18.38 \pm 0.16$|
|statistical parallaxes|$0.76 \pm 0.13$|$18.30 \pm 0.14$|
|main-sequence fitting of Galactic globular clusters|$0.61 \pm 0.07$|$18.45 \pm 0.09$|

He then reconciled the long- and short-distance scale on a distance modulus $\mu_{LMC} = 18.515 \pm 0.085$  

### The up-to-date LMC distance modulus research

From [Pietrzyński et al. 2019](https://ui.adsabs.harvard.edu/abs/2019Natur.567..200P/abstract), they computed the mean distance modulus to LMC $\langle \mu \rangle = 18.477 ± 0.004\,(\text{statistical}) ± 0.026\,(\text{systemical})$

In [ ]:
muLMC   = 18.477
u_muLMC = np.sqrt(0.004**2 + 0.026**2)

#### RR0 distance modulus

In [ ]:
RR0_M = {'gMag':[], 'err_g' : [],
         'rMag':[], 'err_r' : [],
         'iMag':[], 'err_i' : []}

for gmag, err_g in zip(rr0_gri['gmag'], rr0_gri['err_g']):
    g = gmag - muLMC
    ug = np.sqrt( err_g**2 + u_muLMC)
    
    RR0_M['gMag'].append(g)
    RR0_M['err_g'].append(ug)
    
    
for rmag, err_r in zip(rr0_gri['rmag'], rr0_gri['err_r']):
    r = rmag - muLMC
    ur = np.sqrt( err_g**2 + u_muLMC)
    
    RR0_M['rMag'].append(r)
    RR0_M['err_r'].append(ur)
    
    
for imag, err_i in zip(rr0_gri['imag'], rr0_gri['err_i']):
    i = imag - muLMC
    ui = np.sqrt( err_i**2 + u_muLMC)
    
    RR0_M['iMag'].append(i)
    RR0_M['err_i'].append(ui)

#### RR1 distance modulus

In [ ]:
RR1_M = {'gMag':[], 'err_g' : [],
         'rMag':[], 'err_r' : [],
         'iMag':[], 'err_i' : []}

for gmag, err_g in zip(rr1_gri['gmag'], rr1_gri['err_g']):
    g = gmag - muLMC
    ug = np.sqrt( err_g**2 + u_muLMC)
    
    RR1_M['gMag'].append(g)
    RR1_M['err_g'].append(ug)
    
    
for rmag, err_r in zip(rr1_gri['rmag'], rr1_gri['err_r']):
    r = rmag - muLMC
    ur = np.sqrt( err_g**2 + u_muLMC)
    
    RR1_M['rMag'].append(r)
    RR1_M['err_r'].append(ur)
    
    
for imag, err_i in zip(rr1_gri['imag'], rr1_gri['err_i']):
    i = imag - muLMC
    ui = np.sqrt( err_i**2 + u_muLMC)
    
    RR1_M['iMag'].append(i)
    RR1_M['err_i'].append(ui)

## Compare PL relation
### [Ngeow et al. 2022](https://ui.adsabs.harvard.edu/abs/2022AJ....163..239N/abstract)
Equation (1):  
$y = a + b \log P + c[\text{Fe}/\text{H}]$  


PL relation is a special case, for $c$ is setted as $0$

 - RR0  

    $$\begin{aligned}
        M_g &= 0.649 − 0.302 \log P \\
        M_r &= 0.337 − 1.090 \log P \\
        M_i &= 0.243 − 1.432 \log P \\
    \end{aligned}$$

- RRc (RR1)
    $$\begin{aligned}
        M_g &=   0.411 − 0.342 \log P \\
        M_r &= − 0.082 − 1.393 \log P \\
        M_i &= − 0.205 − 1.706 \log P
    \end{aligned}$$

- RR0 + RR1 (combined sample)  
    $ \log P_0 = \log P_1 + 0.127$
    $$\begin{aligned}
        M_g &= 0.801 − 0.032 \log P \\
        M_r &= 0.432 − 0.874 \log P \\
        M_i &= 0.243 − 1.362 \log P \\
    \end{aligned}$$
    
    

### [Narloch et al. 2024](https://ui.adsabs.harvard.edu/abs/2024A%26A...689A.138N/abstract)
Equation (4):  
$M_\lambda = a (\log P - \log P_0) + b$
How to determine the $\log P_0$: [Zgirski et al. (2023)](https://ui.adsabs.harvard.edu/abs/2023ApJ...951..114Z/abstract)
 - RRab (RR0)  
 In Narloch et al. 2024, set $\log P_0 = -0.25$
$$\begin{aligned}
    M_g = -2.242 (\log P - \log P_0) + 0.805 \\
    M_r = -2.528 (\log P - \log P_0) + 0.660 \\
    M_i = -2.799 (\log P - \log P_0) + 0.642 \\
\end{aligned}$$

- RRc (RR1)
In Narloch et al. 2024, set $\log P_0 = -0.45$
$$\begin{aligned}
    M_g = - 1.984 (\log P - \log P_0) + 0.605 \\
    M_r = - 2.266 (\log P - \log P_0) + 0.557 \\
    M_i = - 2.393 (\log P - \log P_0) + 0.598 \\
\end{aligned}$$

- RRab + RRc (combined sample)  
In Narloch et al. 2024, set $\log P_0 = -0.25$
$$\begin{aligned}
    M_g = - 1.500 (\log P - \log P_0) + 0.773 \\
    M_r = - 1.929 (\log P - \log P_0) + 0.634 \\
    M_i = - 2.249 (\log P - \log P_0) + 0.619 \\
\end{aligned}$$

#### RR0 PL relation

In [ ]:
# RR0 period
rr0_logP_g = np.log10(rr0_P['P_g'])
rr0_logP_r = np.log10(rr0_P['P_r'])
rr0_logP_i = np.log10(rr0_P['P_i'])

rr0_Pg_space = np.linspace(min(rr0_logP_g), max(rr0_logP_g), 200)
rr0_Pr_space = np.linspace(min(rr0_logP_r), max(rr0_logP_r), 200)
rr0_Pi_space = np.linspace(min(rr0_logP_i), max(rr0_logP_i), 200)

# Ngeow et al. 2022 
rr0_Mg_Ngeow = 0.649 - 0.302 * rr0_Pg_space
rr0_Mr_Ngeow = 0.337 - 1.090 * rr0_Pr_space
rr0_Mi_Ngeow = 0.243 - 1.432 * rr0_Pi_space

# Narloch et al. 2024
rr0_Mg_Narloch = -2.242 * (rr0_Pg_space - (-0.25)) + 0.805
rr0_Mr_Narloch = -2.528 * (rr0_Pr_space - (-0.25)) + 0.660
rr0_Mi_Narloch = -2.799 * (rr0_Pi_space - (-0.25)) + 0.642

#### RR1 PL relation

In [ ]:
rr1_logP_g = np.log10(rr1_P['P_g'])
rr1_logP_r = np.log10(rr1_P['P_r'])
rr1_logP_i = np.log10(rr1_P['P_i'])

rr1_Pg_space = np.linspace(min(rr1_logP_g), max(rr1_logP_g), 200)
rr1_Pr_space = np.linspace(min(rr1_logP_r), max(rr1_logP_r), 200)
rr1_Pi_space = np.linspace(min(rr1_logP_i), max(rr1_logP_i), 200)

# Ngeow et al. 2022 
rr1_Mg_Ngeow =   0.411 - 0.342 * rr1_Pg_space 
rr1_Mr_Ngeow = - 0.082 - 1.393 * rr1_Pr_space
rr1_Mi_Ngeow = - 0.205 - 1.706 * rr1_Pi_space

# Narloch et al. 2024
rr1_Mg_Narloch = - 1.984 * (rr1_Pg_space - (-0.45)) + 0.605
rr1_Mr_Narloch = - 2.266 * (rr1_Pr_space - (-0.45)) + 0.557
rr1_Mi_Narloch = - 2.393 * (rr1_Pi_space - (-0.45)) + 0.598

#### Combined PL relation

In [ ]:
Crr1_logP_g = rr1_logP_g + 0.127
Crr1_logP_r = rr1_logP_r + 0.127
Crr1_logP_i = rr1_logP_i + 0.127

C_Pg_space = np.linspace(min(Crr1_logP_g), max(rr0_logP_g), 200)
C_Pr_space = np.linspace(min(Crr1_logP_r), max(rr0_logP_r), 200)
C_Pi_space = np.linspace(min(Crr1_logP_i), max(rr0_logP_i), 200)

C_Mg_Ngeow =   0.411 - 0.342 * C_Pg_space 
C_Mr_Ngeow = - 0.082 - 1.393 * C_Pr_space
C_Mi_Ngeow = - 0.205 - 1.706 * C_Pi_space

C_Mg_Narloch = - 1.500 * (C_Pg_space - (-0.25)) + 0.773
C_Mr_Narloch = - 1.929 * (C_Pr_space - (-0.25)) + 0.634
C_Mi_Narloch = - 2.249 * (C_Pi_space - (-0.25)) + 0.619

## Plot

### PL relation for RR0

In [ ]:
fig, axs = plt.subplots(3, 1, figsize=(8, 12), sharex=True)

# Plot g-band
axs[0].plot(rr0_Pg_space, rr0_Mg_Ngeow, color='k', label='Ngeow et al. (2022)')
axs[0].plot(rr0_Pg_space, rr0_Mg_Narloch, color='k', linestyle='dashed', label='Narloch et al. (2024)')
axs[0].scatter(rr0_logP_g, RR0_M['gMag'], color='blue')
axs[0].set_title('PL Relation for RR0')
axs[0].invert_yaxis()  # Reverse y-axis for magnitude
axs[0].set_ylabel(r'$M_g$ [mag]')
axs[0].legend(loc='upper left')

# Plot r-band
axs[1].plot(rr0_Pr_space, rr0_Mr_Ngeow, color='k', label='Ngeow et al. (2022)')
axs[1].plot(rr0_Pr_space, rr0_Mr_Narloch, color='k', linestyle='dashed', label='Narloch et al. (2024)')
axs[1].scatter(rr0_logP_r, RR0_M['rMag'], color='green')
axs[1].invert_yaxis()  # Reverse y-axis for magnitude
axs[1].set_ylabel(r'$M_r$ [mag]')
axs[1].legend(loc='upper left')

# Plot i-band
axs[2].plot(rr0_Pg_space, rr0_Mi_Ngeow, color='k', label='Ngeow et al. (2022)')
axs[2].plot(rr0_Pi_space, rr0_Mi_Narloch, color='k', linestyle='dashed', label='Narloch et al. (2024)')
axs[2].scatter(rr0_logP_i, RR0_M['iMag'], color='red')
axs[2].invert_yaxis()  # Reverse y-axis for magnitude
axs[2].set_ylabel(r'$M_i$ [mag]')
axs[2].set_xlabel(r'$\log_{10}P$ [days]')
axs[2].legend(loc='upper left')

# Adjust layout and display
plt.tight_layout()
# plt.savefig('LMC-RR0-PL_compare-241127.pdf', dpi=300)
plt.show()

### PL relation for RR1

In [ ]:
fig, axs = plt.subplots(3, 1, figsize=(8, 12), sharex=True)

# Plot g-band
axs[0].plot(rr1_Pg_space, rr1_Mg_Ngeow, color='k', label='Ngeow et al. (2022)')
axs[0].plot(rr1_Pg_space, rr1_Mg_Narloch, color='k', linestyle='dashed', label='Narloch et al. (2024)')
axs[0].scatter(rr1_logP_g, RR1_M['gMag'], color='blue')
axs[0].invert_yaxis()  # Reverse y-axis for magnitude
axs[0].set_title('PL Relation for RR1')
axs[0].set_ylabel(r'$M_g$ [mag]')
axs[0].legend(loc='upper left')

# Plot g-band
axs[1].plot(rr1_Pr_space, rr1_Mr_Ngeow, color='k', label='Ngeow et al. (2022)')
axs[1].plot(rr1_Pr_space, rr1_Mr_Narloch, color='k', linestyle='dashed', label='Narloch et al. (2024)')
axs[1].scatter(rr1_logP_r, RR1_M['rMag'], color='green')
axs[1].invert_yaxis()  # Reverse y-axis for magnitude
axs[1].set_ylabel(r'$M_r$ [mag]')
axs[1].legend(loc='upper left')

# Plot g-band
axs[2].plot(rr1_Pg_space, rr1_Mi_Ngeow, color='k', label='Ngeow et al. (2022)')
axs[2].plot(rr1_Pi_space, rr1_Mi_Narloch, color='k', linestyle='dashed', label='Narloch et al. (2024)')
axs[2].scatter(rr1_logP_i, RR1_M['iMag'], color='red')
axs[2].invert_yaxis()  # Reverse y-axis for magnitude
axs[2].set_ylabel(r'$M_i$ [mag]')
axs[2].set_xlabel(r'$\log_{10}P$ [days]')
axs[2].legend(loc='upper left')

# Adjust layout and display
plt.tight_layout()
# plt.savefig('LMC-RR1-PL_compare-241127.pdf', dpi=300)
plt.show()

In [ ]:
fig, axs = plt.subplots(3, 1, figsize=(8, 12), sharex=True)

# Plot g-band
axs[0].plot(C_Pg_space, C_Mg_Ngeow, color='k', label='Ngeow et al. (2022)')
axs[0].plot(C_Pg_space, C_Mg_Narloch, color='k', linestyle='dashed', label='Narloch et al. (2024)')
axs[0].scatter(rr0_logP_g, RR0_M['gMag'], color='blue', label='RR0')
axs[0].scatter(Crr1_logP_g, RR1_M['gMag'], color='darkorange', label='RR1')
axs[0].invert_yaxis()  # Reverse y-axis for magnitude
axs[0].set_title('PL Relation for RR0+RR1')
axs[0].set_ylabel(r'$M_g$ [mag]')
axs[0].legend(loc='upper left')

# Plot g-band
axs[1].plot(C_Pr_space, C_Mr_Ngeow, color='k', label='Ngeow et al. (2022)')
axs[1].plot(C_Pr_space, C_Mr_Narloch, color='k', linestyle='dashed', label='Narloch et al. (2024)')
axs[1].scatter(rr0_logP_r, RR0_M['rMag'], color='green', label='RR0')
axs[1].scatter(Crr1_logP_r, RR1_M['rMag'], color='deeppink', label='RR1')
axs[1].invert_yaxis()  # Reverse y-axis for magnitude
axs[1].set_ylabel(r'$M_r$ [mag]')
axs[1].legend(loc='upper left')

# Plot g-band
axs[2].plot(C_Pg_space, C_Mi_Ngeow, color='k', label='Ngeow et al. (2022)')
axs[2].plot(C_Pi_space, C_Mi_Narloch, color='k', linestyle='dashed', label='Narloch et al. (2024)')
axs[2].scatter(rr0_logP_i, RR0_M['iMag'], color='red', label='RR0')
axs[2].scatter(Crr1_logP_i, RR1_M['iMag'], color='darkturquoise', label='RR1')
axs[2].invert_yaxis()  # Reverse y-axis for magnitude
axs[2].set_ylabel(r'$M_i$ [mag]')
axs[2].set_xlabel(r'$\log_{10}P$ [days]')
axs[2].legend(loc='upper left')

# Adjust layout and display
plt.tight_layout()
# plt.savefig('LMC-Combine-PL_compare-241127.pdf', dpi=300)
plt.show()